In [88]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_tweedie_deviance

In [54]:

data = pd.read_csv('/Users/arthurrastelli/Desktop/Assignment/Assignment.csv')

data = data.rename(columns= {'duree': 'expo','nbrtotan': 'freq','nbrtotc': 'nclaims'})
data["sev"] = data['chargtot']/data['nclaims']
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163657 entries, 0 to 163656
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   AGEPH     163657 non-null  int64  
 1   CODPOSS   163657 non-null  int64  
 2   expo      163657 non-null  float64
 3   lnexpo    163657 non-null  float64
 4   nclaims   163657 non-null  int64  
 5   freq      163657 non-null  float64
 6   chargtot  163657 non-null  float64
 7   agecar    163657 non-null  object 
 8   sexp      163657 non-null  object 
 9   fuelc     163657 non-null  object 
 10  split     163657 non-null  object 
 11  usec      163657 non-null  object 
 12  fleetc    163657 non-null  object 
 13  sportc    163657 non-null  object 
 14  coverp    163657 non-null  object 
 15  powerc    163657 non-null  object 
 16  sev       18345 non-null   float64
dtypes: float64(5), int64(3), object(9)
memory usage: 21.2+ MB


,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,NaN
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,NaN
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,NaN
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,NaN


Bin CODPOSS

In [55]:
def categorize_postal_code(code):
    if 1000 <= code <= 1299:
        return 'BHG'
    elif 1300 <= code <= 1499:
        return 'WB'
    elif 1500 <= code <= 1999 or 3000 <= code <= 3499:
        return 'VB'
    elif 2000 <= code <= 2999:
        return 'ANT'
    elif 3500 <= code <= 3999:
        return 'LIM'
    elif 4000 <= code <= 4999:
        return 'LUI'
    elif 5000 <= code <= 5999:
        return 'NAM'
    elif 6000 <= code <= 6599 or 7000 <= code <= 7999:
        return 'HEN'
    elif 6600 <= code <= 6999:
        return 'LUX'
    elif 8000 <= code <= 8999:
        return 'WV'
    elif 9000 <= code <= 9999:
        return 'OV'
    else:
        return 'Unknown'
    
data['region'] = data['CODPOSS'].apply(categorize_postal_code)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,region
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,NaN,BHG
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,BHG
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,NaN,BHG
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,NaN,BHG
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,NaN,BHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163652,50,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Once,Private,No,No,MTPL+,<66,NaN,OV
163653,86,9960,1.000000,0.000000,0,0.000000,0.000000,6-10,Male,Petrol,Once,Private,No,No,MTPL,<66,NaN,OV
163654,64,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Twice,Private,No,No,MTPL,<66,NaN,OV
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,OV


GENERALIZED LINEAR MODELS: FREQUENCY

In [56]:
bins = [0,19,29,39,49,59,69,79,89,99]
labels=['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']

data['age_bins'] = pd.cut(data['AGEPH'],bins=bins,labels=labels,right=False)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,region,age_bins
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110,NaN,BHG,60-69
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,BHG,20-29
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66,NaN,BHG,50-59
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110,NaN,BHG,30-39
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66,NaN,BHG,30-39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163652,50,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Once,Private,No,No,MTPL+,<66,NaN,OV,50-59
163653,86,9960,1.000000,0.000000,0,0.000000,0.000000,6-10,Male,Petrol,Once,Private,No,No,MTPL,<66,NaN,OV,80-89
163654,64,9960,1.000000,0.000000,0,0.000000,0.000000,2-5,Male,Petrol,Twice,Private,No,No,MTPL,<66,NaN,OV,60-69
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,OV,50-59


In [57]:
formula1 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [58]:
glm_fit1 = smf.glm(formula=formula1, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit1.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               163657
Model:                            GLM   Df Residuals:                   163625
Model Family:                 Poisson   Df Model:                           31
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -91544.
Date:                Mon, 20 May 2024   Deviance:                   1.4337e+05
Time:                        22:47:09   Pearson chi2:                 5.66e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03074
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [59]:
data = data[data['region'] != "WB"]

In [60]:
formula2 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [61]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               156977
Model:                            GLM   Df Residuals:                   156946
Model Family:                 Poisson   Df Model:                           30
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -88068.
Date:                Mon, 20 May 2024   Deviance:                   1.3813e+05
Time:                        22:51:33   Pearson chi2:                 5.66e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03126
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [64]:
data = data[data['region'] != "LUI"]

In [65]:
formula2 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [66]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               135168
Model:                            GLM   Df Residuals:                   135138
Model Family:                 Poisson   Df Model:                           29
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -77511.
Date:                Mon, 20 May 2024   Deviance:                   1.2229e+05
Time:                        22:55:24   Pearson chi2:                 5.58e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03330
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [67]:
data = data[data['region'] != "ANT"]

In [68]:
formula2 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [69]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               115384
Model:                            GLM   Df Residuals:                   115355
Model Family:                 Poisson   Df Model:                           28
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -66597.
Date:                Mon, 20 May 2024   Deviance:                   1.0506e+05
Time:                        22:56:24   Pearson chi2:                 3.91e+07
No. Iterations:                     9   Pseudo R-squ. (CS):            0.03738
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

REMOVE USEC

In [70]:
formula2 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [71]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               115384
Model:                            GLM   Df Residuals:                   115356
Model Family:                 Poisson   Df Model:                           27
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -66597.
Date:                Mon, 20 May 2024   Deviance:                   1.0506e+05
Time:                        22:57:25   Pearson chi2:                 3.90e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.03738
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [72]:
data = data[data['powerc'] != ">110"]

In [73]:
formula2 = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [74]:
glm_fit2 = smf.glm(formula=formula2, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit2.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               114253
Model:                            GLM   Df Residuals:                   114226
Model Family:                 Poisson   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -66082.
Date:                Mon, 20 May 2024   Deviance:                   1.0429e+05
Time:                        22:59:10   Pearson chi2:                 3.90e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.03755
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [75]:
data = data[data['region'] != "VB"]

In [78]:
formula = '''
freq ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(fuelc, Treatment(reference="Petrol")) + 
C(split, Treatment(reference="Once")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [79]:
glm_fit = smf.glm(formula=formula, data=data, exposure=data['expo'], family = sm.families.Poisson(link=sm.families.links.log())).fit()
print(glm_fit.summary())

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:               100690
Model:                            GLM   Df Residuals:                   100664
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -59530.
Date:                Mon, 20 May 2024   Deviance:                       94310.
Time:                        23:01:40   Pearson chi2:                 3.82e+07
No. Iterations:                    10   Pseudo R-squ. (CS):            0.04081
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [82]:
prediction = glm_fit.predict(data)
MSE = np.mean((data['freq']-prediction)**2)
MSE

1.1998817765926428

GENERALIZED LINEAR MODELS: SEVERITY

In [104]:
data = pd.read_csv('/Users/arthurrastelli/Desktop/Assignment/Assignment.csv')

data = data.rename(columns= {'duree': 'expo','nbrtotan': 'freq','nbrtotc': 'nclaims'})
data = data[data['nclaims'] != 0]
data["sev"] = data['chargtot']/data['nclaims']
data["logsev"] = np.log(data["sev"])
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 18345 entries, 1 to 163656
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AGEPH     18345 non-null  int64  
 1   CODPOSS   18345 non-null  int64  
 2   expo      18345 non-null  float64
 3   lnexpo    18345 non-null  float64
 4   nclaims   18345 non-null  int64  
 5   freq      18345 non-null  float64
 6   chargtot  18345 non-null  float64
 7   agecar    18345 non-null  object 
 8   sexp      18345 non-null  object 
 9   fuelc     18345 non-null  object 
 10  split     18345 non-null  object 
 11  usec      18345 non-null  object 
 12  fleetc    18345 non-null  object 
 13  sportc    18345 non-null  object 
 14  coverp    18345 non-null  object 
 15  powerc    18345 non-null  object 
 16  sev       18345 non-null  float64
 17  logsev    18345 non-null  float64
dtypes: float64(6), int64(3), object(9)
memory usage: 2.7+ MB


,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290


Bin the CODPOSS

In [105]:
def categorize_postal_code(code):
    if 1000 <= code <= 1299:
        return 'BHG'
    elif 1300 <= code <= 1499:
        return 'WB'
    elif 1500 <= code <= 1999 or 3000 <= code <= 3499:
        return 'VB'
    elif 2000 <= code <= 2999:
        return 'ANT'
    elif 3500 <= code <= 3999:
        return 'LIM'
    elif 4000 <= code <= 4999:
        return 'LUI'
    elif 5000 <= code <= 5999:
        return 'NAM'
    elif 6000 <= code <= 6599 or 7000 <= code <= 7999:
        return 'HEN'
    elif 6600 <= code <= 6999:
        return 'LUX'
    elif 8000 <= code <= 8999:
        return 'WV'
    elif 9000 <= code <= 9999:
        return 'OV'
    else:
        return 'Unknown'
    
data['region'] = data['CODPOSS'].apply(categorize_postal_code)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev,region
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693,BHG
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895,BHG
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723,BHG
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552,BHG
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290,BHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163624,43,9700,1.000000,0.000000,2,2.000000,4270.511330,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,2135.255665,7.666342,OV
163636,58,9820,1.000000,0.000000,1,1.000000,145.017712,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,145.017712,4.976856,OV
163642,27,9890,1.000000,0.000000,1,1.000000,271.220305,2-5,Male,Petrol,Once,Private,No,No,MTPL+++,<66,271.220305,5.602931,OV
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,8.064006,OV


BIN THE AGEPH VARIABLE

In [106]:
bins = [0,19,29,39,49,59,69,79,89,99]
labels=['0-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']

data['age_bins'] = pd.cut(data['AGEPH'],bins=bins,labels=labels,right=False)
data

,AGEPH,CODPOSS,expo,lnexpo,nclaims,freq,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc,sev,logsev,region,age_bins
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,155.974606,5.049693,BHG,20-29
12,72,1050,1.000000,0.000000,1,1.000000,1426.379342,2-5,Female,Petrol,Once,Private,No,No,MTPL+++,66-110,1426.379342,7.262895,BHG,70-79
15,28,1060,0.956164,-0.044825,2,2.091691,1576.354924,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110,788.177462,6.669723,BHG,20-29
20,52,1070,0.452055,-0.793952,3,6.636364,8885.396345,6-10,Female,Petrol,Monthly,Private,No,Yes,MTPL,>110,2961.798782,7.993552,BHG,50-59
22,56,1082,1.000000,0.000000,1,1.000000,619.733812,0-1,Female,Petrol,Once,Private,No,No,MTPL+,66-110,619.733812,6.429290,BHG,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163624,43,9700,1.000000,0.000000,2,2.000000,4270.511330,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,2135.255665,7.666342,OV,40-49
163636,58,9820,1.000000,0.000000,1,1.000000,145.017712,>10,Male,Petrol,Once,Private,No,No,MTPL,<66,145.017712,4.976856,OV,50-59
163642,27,9890,1.000000,0.000000,1,1.000000,271.220305,2-5,Male,Petrol,Once,Private,No,No,MTPL+++,<66,271.220305,5.602931,OV,20-29
163655,58,9970,1.000000,0.000000,1,1.000000,3177.994988,>10,Male,Petrol,Once,Private,Yes,No,MTPL,<66,3177.994988,8.064006,OV,50-59


Variables are removed in order

In [117]:
formula = '''
logsev ~
C(age_bins, Treatment(reference="20-29")) + 
C(agecar, Treatment(reference="6-10")) + 
C(sexp, Treatment(reference="Male")) + 
C(split, Treatment(reference="Once")) + 
C(usec, Treatment(reference="Private")) + 
C(fleetc, Treatment(reference="No")) + 
C(sportc, Treatment(reference="No")) + 
C(powerc, Treatment(reference="<66")) +
C(region, Treatment(reference="HEN"))
'''

In [118]:
glm_fit = smf.glm(formula=formula, data=data, family = sm.families.Gamma(link=sm.families.links.Log())).fit()
print(glm_fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 logsev   No. Observations:                12742
Model:                            GLM   Df Residuals:                    12714
Model Family:                   Gamma   Df Model:                           27
Link Function:                    Log   Scale:                        0.057499
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Mon, 20 May 2024   Deviance:                       1049.6
Time:                        23:27:27   Pearson chi2:                     731.
No. Iterations:                     9   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                                                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:812: RuntimeWarning: invalid value encountered in log
  ll_obs -= special.gammaln(weight_scale) + np.log(endog)


In [119]:
predictions = glm_fit.predict()
diff = data['freq']
mse = np.mean(diff)
mse

1.3824725127667008

In [121]:
columns = np.array(data.columns)
columns

array(['AGEPH', 'CODPOSS', 'expo', 'lnexpo', 'nclaims', 'freq',
       'chargtot', 'agecar', 'sexp', 'fuelc', 'split', 'usec', 'fleetc',
       'sportc', 'coverp', 'powerc', 'sev', 'logsev', 'region',
       'age_bins'], dtype=object)

In [122]:

list = data.iloc[1,:]

freq_cal = glm_fit8.predict(list)
severity_cal = glm_fit_sev1.predict(list)
pure_premium = freq_cal * severity_cal * expo

NameError: name 'expo' is not defined

In [123]:
def premium_calculator(list,expo):
    columns= list
    freq_cal = glm_fit8.predict(list)
    severity_cal = glm_fit_sev1.predict(list)

    pure_premium = freq_cal * severity_cal * expo
    return pure_premium

In [124]:
data.iloc[1,:]

AGEPH                72
CODPOSS            1050
expo                1.0
lnexpo              0.0
nclaims               1
freq                1.0
chargtot    1426.379342
agecar              2-5
sexp             Female
fuelc            Petrol
split              Once
usec            Private
fleetc               No
sportc               No
coverp          MTPL+++
powerc           66-110
sev         1426.379342
logsev         7.262895
region              BHG
age_bins          70-79
Name: 12, dtype: object

In [125]:
premium_calculator(data.iloc[1,:],1)

12    290.732894
dtype: float64